In [5]:
# Participant Activity Duration Summary
import os
import sys
import pandas as pd
from pathlib import Path

# Configuration
DATA_DIR = Path('../Data/Experiment_Data/2_PreprocessDataset')
OUTPUT_CSV = Path('../Data/Experiment_Data/participant_activity_with_other.csv')

if __name__ == '__main__':
    records = []
    session_times = {}

    # Iterate participants
    for subdir in sorted(DATA_DIR.iterdir()):
        if not subdir.is_dir():
            continue
        pid = subdir.name
        anno_file = subdir / f"{pid}_Annotation_processed.csv"
        if not anno_file.exists():
            continue

        df = pd.read_csv(anno_file)
        df = df.sort_values('UnixTime_s')

        # Compute total session duration
        start_ts = df.loc[df.Event=='Session Start', 'UnixTime_s'].min()
        stop_ts  = df.loc[df.Event=='Session Stop',  'UnixTime_s'].max()
        session_times[pid] = (stop_ts - start_ts) / 60.0

        # Aggregate per-activity durations
        ongoing = {}
        for _, row in df.iterrows():
            act, ev, ts = row.Activity, row.Event, row.UnixTime_s
            if ev == 'Start':
                ongoing[act] = ts
            elif ev == 'End' and act in ongoing:
                duration = (ts - ongoing.pop(act)) / 60.0
                records.append({'Participant': pid, 'Activity': act, 'Duration_min': duration})

    # Create summary DataFrame
    rec_df = pd.DataFrame(records)
    summary = (
        rec_df.groupby(['Participant','Activity'], as_index=False)
              .agg(Total_Duration_min=('Duration_min','sum'))
    )

    # Pivot to wide format
    pivot = summary.pivot(index='Participant', columns='Activity', values='Total_Duration_min').fillna(0)

    # Compute 'Other' as remaining session time
    pivot['Other'] = pivot.index.to_series().map(session_times) - pivot.sum(axis=1)

    # Save to CSV
    pivot.to_csv(OUTPUT_CSV)
    print(f"Saved participant activity summary (with 'Other') to {OUTPUT_CSV.resolve()}")

Saved participant activity summary (with 'Other') to D:\Project\HCAR\Data\Experiment_Data\participant_activity_with_other.csv
